# Import libraries

In [ ]:
import os
import cv2
import json
import time
from tqdm import tqdm

# basic DL tools
import numpy as np
import pandas as pd     # dataframe
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from PIL import Image

# PyTorch
import torch
from torch import Tensor    # inference
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

# for classification model
from skimage import io
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import RandomSampler
import torchvision.models as models     # torchvision에서 models import했는데 또?
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from sklearn.metrics import precision_score, recall_score   # metrics

# for regression model
import torch.utils.data as data


# Set Directory

In [ ]:
train_dir = os.getcwd() + '/DatasetD84/Training'    
valid_dir = os.getcwd() + '/DatasetD84/Validation'    
test_dir = os.getcwd() + '/DatasetD84/Test'    

# root_dir = '/home/bmegpu01/DatasetD841/Training'

# Metrics

In [ ]:
def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    # box = (x, y, w, h)
#     print(box1.shape, box2.shape)

    box1_area = (box1[2]*box1[3])
    box2_area = (box2[2]*box2[3])
    box_1 = [box1[0], box1[1], box1[0]+box1[2], box1[1]+box1[3]]
    box_2 = [box2[0], box2[1], box2[0]+box2[2], box2[1]+box2[3]]

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box_1[0], box_2[0])
    y1 = max(box_1[1], box_2[1])
    x2 = min(box_1[2], box_2[2])
    y2 = min(box_1[3], box_2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou

def batch_IoU(y_hat, y, images):
    score = []
    num_batch = y_hat.shape[0]
    for num in range(num_batch):
        c, width, height = images[num].shape
        scale = torch.tensor([width, height, width, height])
        y_hat = y_hat * scale
        y = y * scale
        score.append(IoU(y_hat[num], y[num]))
    return score

In [ ]:
# for classification
def calc_accuracy(true,pred):
    pred = F.softmax(pred, dim = 1)
    true = torch.zeros(pred.shape[0], pred.shape[1]).scatter_(1, true.unsqueeze(1), 1.)
    acc = (true.argmax(-1) == pred.argmax(-1)).float().detach().numpy()
    acc = float((100 * acc.sum()) / len(acc))
    return round(acc, 4)

In [ ]:
# for classification
def rp_for_class(y_pre, y_true):
    y_pre_fclass = [[],[],[],[],[]]
    y_true_fclass = [[],[],[],[],[]]
    
    for i in range(len(y_true)):
        for num in range(5):
            if y_true[i] == num:
                y_pre_fclass[num].append(y_pre[i])
                y_true_fclass[num].append((y_true[i]))
            
    return y_pre_fclass, y_true_fclass

# Regression

### regression training preset

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
reg_model = models.resnet34(pretrained=True)
reg_model.fc = torch.nn.Linear(512,4)
reg_model = reg_model.to(device)

In [ ]:
#parameter

max_iterations =  512
eval_num = 32

loss_function = torch.nn.HuberLoss(reduction='mean', delta=1.0)
optimizer = torch.optim.Adam(reg_model.parameters(), 
                            lr=3e-3)

global_step = 0
dice_val_best = 0.0
global_step_best = 0
epoch_loss_values = []
metric_values = []

### Regression dataset

In [ ]:
class BurnDataset(Dataset):
    def __init__(self, root_dir, transform = None):
        super().__init__()
        
        self.root_dir = root_dir
        self.img_root_dir = root_dir+'/jpg'
        self.img_flist = []
        
        self.class_name = [x for x in os.listdir(self.img_root_dir) if not x.startswith('@') and not x.startswith('.') and not x.startswith('(')]

        self.transform = transform
        
        for c in self.class_name:
            imgls = os.listdir(os.path.join(self.img_root_dir,c))
            imgls = [os.path.join(self.img_root_dir,c,x) for x in imgls if x.endswith('.jpg')]
            self.img_flist.extend(imgls)
        
    def __len__(self):
        return len(self.img_flist)
        
    def __getitem__(self, idx):
        
        imgpth = self.img_flist[idx]
        jsonpth = self.img_flist[idx].replace('jpg', 'json')
        
#         print(jsonpth)
        # image
#         np_img = np.fromfile(imgpth, np.uint8)
#         image = cv2.imdecode(np_img, cv2.IMREAD_COLOR).astype(np.float32)
#         image = image.transpose(2, 0, 1)
        
        with open(imgpth, 'rb') as f:
            image = Image.open(f).convert('RGB')
            x_, y_ = image.size
        if self.transform:
            image = self.transform(image)
        
        #bbox
        with open (jsonpth, "r") as f:
            data = json.load(f)
            
        labels = torch.LongTensor([int(list(data['annotations'].keys())[0])])
        stage_id = list(data['annotations'].keys())[0]
        boxes = data['annotations'][stage_id]['bbox']
        
        c, width, height = image.shape
        
        x = boxes[0]/width
        y = boxes[1]/height
        
        w = boxes[2]/width
        h = boxes[3]/height
        
#         x = boxes[0][0]/width
#         y = boxes[0][1]/height
        
#         w = boxes[0][2]/width
#         h = boxes[0][3]/height
        
        bbox = torch.FloatTensor([x, y, w, h])


        return image, bbox, labels
        

In [ ]:
from torchvision import transforms, models

tr_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
ts_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

train_set = BurnDataset(train_dir, transform = tr_transform)
train_loader = DataLoader(train_set,
                              batch_size=32,
                              shuffle=True,
                              num_workers=24,
                              pin_memory=True)

val_set = BurnDataset(valid_dir, transform = val_transform)
valid_loader = DataLoader(val_set,
                          batch_size=1,
                          shuffle=False,
                          num_workers=24,
                          pin_memory=True)

test_set = BurnDataset(test_dir, transform = ts_transform) # 수정 부분
test_loader = DataLoader(test_set,
                        batch_size=1,
                        shuffle=False,
                        num_workers=24,
                        pin_memory=True)


### regression train

In [ ]:
def train(model, global_step, train_loader, dice_val_best, global_step_best):
    model.train()
    epoch_loss = 0
    step = 0
    epoch_iterator = tqdm(
        train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True
    )
    for step, (images, boxes, labels) in enumerate(epoch_iterator):
        step += 1
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)
                
        out_bb = model(images)
        
        loss = loss_function(out_bb, boxes)
        loss.backward()
        epoch_loss += loss.item()
        
        optimizer.step()
        optimizer.zero_grad()
        
        epoch_iterator.set_description(
            "Training (%d / %d Steps) (loss=%2.5f)" % (global_step, max_iterations, epoch_loss/step)
        )
        
    if global_step % eval_num == 0 or global_step == max_iterations:
        epoch_iterator_val = tqdm(
            valid_loader, desc="Validate (X / X Steps) (IoU=X.X)", dynamic_ncols=True
        )
        dice_val = validation(model, epoch_iterator_val)
        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        metric_values.append(dice_val)
        plt.subplot(121)
        plt.plot(epoch_loss_values)
        plt.title(f'step #: {global_step}')
        plt.subplot(122)
        plt.plot(metric_values)
        plt.show()

        if dice_val > dice_val_best:
            dice_val_best = dice_val
            global_step_best = global_step
            torch.save(
                model.state_dict(), os.path.join( f"tmp_best_regression_aug_cv.pth")
            )
            print(
                "Model Was Saved ! Current Best Avg. IoU: {} Current Avg. IoU: {}".format(
                    dice_val_best, dice_val
                )
            )
        else:
            print(
                "Model Was Not Saved ! Current Best Avg. IoU: {} Current Avg. IoU: {}".format(
                    dice_val_best, dice_val
                )
            )
    
    global_step += 1

#     scheduler.step()

    return global_step, dice_val_best, global_step_best

### regression Validation

In [ ]:
def validation(model, epoch_iterator_val):
    model.eval()
    dice_vals = list()
    with torch.no_grad():
        for step, (images, boxes, labels) in enumerate(epoch_iterator_val):
            step += 1
            val_images = images.to(device)
            val_boxes = boxes.to(device)
            val_labels =  labels.to(device)

            val_out_bb = model(val_images)    
            val_loss = loss_function(val_out_bb, val_boxes)
            
#             print(val_boxes, val_out_bb)
            
            # acc value
            # def iou            
            iou_acc = batch_IoU(val_boxes.cpu(), val_out_bb.detach().cpu(), val_images.detach().cpu())
            dice_vals.append(iou_acc)
            epoch_iterator_val.set_description(
                "Validate (%d / %d Steps) (IoU=%2.5f)" % (global_step, max_iterations, np.mean(iou_acc))
            )
                  
    mean_dice_val = np.mean(dice_vals)
    return mean_dice_val

### regression training

In [ ]:
while global_step < max_iterations:
    global_step, dice_val_best, global_step_best = train(
        reg_model, global_step, train_loader, dice_val_best, global_step_best
    )

In [ ]:
optimizer = torch.optim.Adam(reg_model.parameters(), 
                             lr=(5e-3)*0.8)

while global_step < 2*max_iterations:
    global_step, dice_val_best, global_step_best = train(
        reg_model, global_step, train_loader, dice_val_best, global_step_best
    )

In [ ]:
optimizer = torch.optim.Adam(reg_model.parameters(), 
                             lr=(5e-3)*0.8*0.8)

while global_step < 4*max_iterations:
    global_step, dice_val_best, global_step_best = train(
        reg_model, global_step, train_loader, dice_val_best, global_step_best
    )

In [ ]:
optimizer = torch.optim.Adam(reg_model.parameters(), 
                             lr=(5e-3)*0.8*0.8*0.8)

while global_step < 6*max_iterations:
    global_step, dice_val_best, global_step_best = train(
        reg_model, global_step, train_loader, dice_val_best, global_step_best
    )

# Classification

### classification training preset

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

cls_model = models.resnet34()
cls_model.fc = torch.nn.Linear(512,5)
cls_model = cls_model.to(device)

In [ ]:
optimizer = torch.optim.Adam(cls_model.parameters(), lr = 5e-3)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.75)
criterion = nn.CrossEntropyLoss()

In [ ]:
max_iterations = 512
eval_num = 32

global_step = 0
dice_val_best = 0.0
global_step_best = 0
epoch_loss_values = []
metric_values = []


### classification loader

In [ ]:
from torchvision import transforms

tr_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
val_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
ts_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# root_dir = os.getcwd() + '/drive/MyDrive/DatasetD84/'

train_set = BurnDataset(root_dir + 'Training', transform= tr_transforms)
train_loader = DataLoader(train_set,
                              batch_size=32,
                              shuffle=True,
                              num_workers=24,
                              pin_memory=True)

valid_set = BurnDataset(root_dir + 'Validation', transform= val_transforms)
valid_loader = DataLoader(valid_set,
                          batch_size=1,
                          shuffle=False,
                          num_workers=24,
                          pin_memory = True)


test_set = BurnDataset(root_dir + 'Test', transform = ts_transforms)
test_loader = DataLoader(test_set,
                        batch_size=1,
                        shuffle=False,
                        num_workers=24,
                        pin_memory=True)

### classification train def

In [ ]:
def train(model, global_step, train_loader, dice_val_best, global_step_best):
    model.train()
    epoch_loss = 0
    step = 0
    epoch_iterator = tqdm(
        train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True
    )
    for step, (images, boxes, labels) in enumerate(epoch_iterator):
        step += 1
        images = images.to(device)
#         boxes = boxes[0].to(device)
        labels = labels.to(device)
                
        out = cls_model(images)
        labels = labels.squeeze(dim=-1)
#         print(labels)
        
        loss = criterion(out, labels)
        loss.backward()
        epoch_loss += loss.item()
        
        optimizer.step()
        optimizer.zero_grad()
        
        epoch_iterator.set_description(
            "Training (%d / %d Steps) (loss=%2.5f)" % (global_step, max_iterations, loss)
        )
    if global_step % eval_num == 0 or global_step == max_iterations:
        epoch_iterator_val = tqdm(
            valid_loader, desc="Validate (X / X Steps) (loss=X.X)", dynamic_ncols=True
        )
        dice_val = validation(cls_model, epoch_iterator_val)
        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        metric_values.append(dice_val)
        plt.subplot(121)
        plt.plot(epoch_loss_values)
        plt.title(f'step #: {global_step}')
        plt.subplot(122)
        plt.plot(metric_values)
        plt.show()

        if dice_val > dice_val_best:
            dice_val_best = dice_val
            global_step_best = global_step
            torch.save(
                cls_model.state_dict(), os.path.join( f"tmp_best_classification_aug.pth")
            )
            print(
                "Model Was Saved ! Current Best Avg. loss: {} Current Avg. loss: {}".format(
                    dice_val_best, dice_val
                )
            )
        else:
            print(
                "Model Was Not Saved ! Current Best Avg. loss: {} Current Avg. loss: {}".format(
                    dice_val_best, dice_val
                )
            )
    global_step += 1

#    scheduler.step()

    return global_step, dice_val_best, global_step_best

### classification validation def

In [ ]:
def validation(model, epoch_iterator_val):
    model.eval()
    dice_vals = list()
    with torch.no_grad():
        for step, (images, boxes, labels) in enumerate(epoch_iterator_val):
            step += 1
            val_images = images.to(device)
            # val_boxes = boxes[0].to(device)
            val_labels =  labels.to(device)
            val_labels = val_labels.squeeze(dim=-1)
            val_out = cls_model(val_images)    
            val_loss = criterion(val_out, val_labels)
            
#             print(val_boxes, val_out_bb)
            
            # acc value
            # def iou

            acc_val = calc_accuracy(val_labels.cpu(), val_out.cpu())
            dice_vals.append(acc_val)
            
            epoch_iterator_val.set_description(
                    "Validate (%d / %d Steps) (Acc=%2.5f)" % (global_step, max_iterations, acc_val)
                )   # 여기의 iou_acc는 어디서 나타난 것...?
                
    mean_dice_val = np.mean(dice_vals)
    return mean_dice_val

### classification training

In [ ]:
while global_step < max_iterations:
    global_step, dice_val_best, global_step_best = train(
        cls_model ,global_step, train_loader, dice_val_best, global_step_best
    )

In [ ]:
optimizer = torch.optim.Adam(cls_model.parameters(), 
                             lr=(5e-3)*0.8)

while global_step < 2*max_iterations:
    global_step, dice_val_best, global_step_best = train(
        cls_model, global_step, train_loader, dice_val_best, global_step_best
    )

In [ ]:
optimizer = torch.optim.Adam(cls_model.parameters(), 
                             lr=(5e-3)*0.8*0.8)

while global_step < 4*max_iterations:
    global_step, dice_val_best, global_step_best = train(
         cls_model,global_step, train_loader, dice_val_best, global_step_best
    )

In [ ]:
optimizer = torch.optim.Adam(cls_model.parameters(), 
                             lr=(5e-3)*0.8*0.8*0.8)

while global_step < 6*max_iterations:
    global_step, dice_val_best, global_step_best = train(
         cls_model,global_step, train_loader, dice_val_best, global_step_best
    )

### classification Test

In [ ]:
def test(epoch_iterator_test):
    reg_model.eval()
    cls_model.eval()
    
    dice_vals = list()
    dice_vals_labels = [[], [], [], [], []]
    
    y_pre, y_true = [], []
    inference_time = [[], [], [], [], []]
    
#     mapiou = [[], [], [], [], []]
#     mapiou_fp = [[], [], [], [], []]
    
    with torch.no_grad():
        for step, (images, boxes, labels) in enumerate(epoch_iterator_test):
            step += 1
            
            # regression
            val_images = images.to(device)
            val_boxes = boxes.to(device)
            val_labels =  labels.to(device)

            val_out_bb = reg_model(val_images)    
            val_loss = loss_function(val_out_bb, val_boxes)
            
            iou_acc = batch_IoU(val_boxes.cpu(), val_out_bb.detach().cpu(), val_images.detach().cpu())
            print(iou_acc)
            bbox_plot(val_boxes.cpu(), val_out_bb.detach().cpu(), iou_acc)
            num_batch = len(labels)
            for num in range(num_batch):
                dice_vals_labels[val_labels.cpu()].append(iou_acc[num])
                
                    
                
            dice_vals.append(iou_acc)
            
            # classification
            val_images2 = images.to(device)
            val_labels2 =  labels.to(device)
            val_labels2 = val_labels2.squeeze(dim=-1)
            
            # :TODO: predict time 
            start = time.time()
            val_out_cls = cls_model(val_images2)   
            preds_1d = val_out_cls.detach().cpu().flatten()
            best_pred_class = [np.argmax(preds_1d)]
            predict_time = time.time() - start
            
            for num in range(5):
                if val_labels.cpu() == num:
                    inference_time[num].append(predict_time)

            y_pre.append(*best_pred_class)
            y_true.append(*val_labels.cpu())
            y_pre, y_true = rp_for_class(y_pre, y_true)
            
#             for num in range(num_batch):
#                 dice_vals_labels[val_labels.cpu()].append(iou_acc[num])
                
#                 if iou_acc[num] >= 0.5:
#                     mapiou[val_labels.cpu()].append(*best_pred_class)
#                     mapiou_fp[val_labels.cpu()].append(*val_labels.cpu())
            
            
            
            if step%100==0:
                epoch_iterator_test.set_description(
                    "Test (%d / %d Steps) (IoU=%2.5f)" % (global_step, 10.0, np.mean(iou_acc))
                )
                  
    mean_dice_val = np.mean(dice_vals)
#     mean_dice_val_labels = np.mean(dice_vals_labels)
    for i in range(5):
        inference_time[i] = np.mean(inference_time[i])
        dice_vals_labels[i] = np.mean(dice_vals_labels[i])
    return mean_dice_val, dice_vals_labels, y_pre, y_true,inference_time, mapiou, mapiou_fp

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def bbox_plot(a, b, iouval):
    
    rect = [a[0],
            b[0]
           ]
    
    plt.style.use('default')
    plt.rcParams['figure.figsize'] = (5, 4)
    plt.rcParams['font.size'] = 12
    
    fig, ax = plt.subplots()
    color = ['green', 'red']
    for idx, (x, y, w, h) in enumerate(rect):
        ax.add_patch(
            patches.Rectangle(
                (x, y), w, h,
                edgecolor= color[idx],
                facecolor=None,
                linewidth=0.5,
                fill=False))
    plt.title(f'{iouval}')

    plt.show()

## result

In [ ]:
epoch_iterator_test = tqdm(test_loader, desc="Test (X / X Steps) (IoU=X.X)", dynamic_ncols=True)

score, score_labels, y_pre, y_true, inference_time, mapiou, mapiou_fp = test(epoch_iterator_test)

In [ ]:
precision = []
recall = []

# li_mapiou = []

for i in range(5):
    pre_val = precision_score(y_true[i], y_pre[i], labels=[0,1,2,3,4], average='micro')
    recall_val = recall_score(y_true[i], y_pre[i], labels=[0,1,2,3,4], average='micro')
    
#     map_val = precision_score(mapiou[i], mapiou_fp[i], labels=[0,1,2,3,4], average='micro')
#     li_mapiou.append(map_val)
    
    precision.append(pre_val)
    recall.append(recall_val)

In [ ]:
print('precision : ', precision)
print('recall : ', recall)
print('inference_time : ', inference_time)
print('score_labels : ', score_labels)

In [ ]:
data1 = {'index' : ['1 degree', '2 degree (superficial)', '2 degree (deep)', '3 degree', '4 degree'],
        'mAP@IoU(0.5)' : ['','','','','',],
        'IoU' : score_labels,
        'Precision' : precision,
        'Recall' : recall,
        'Inference time(s)' : inference_time
        }

data2 = {
    'index' : ['All class'],
    'mAP@IoU(0.5)' : [''],
    'IoU' : score,
    'Precision' : np.mean(precision),
    'Recall' : np.mean(recall),
    'Inference time(s)' : np.mean(inference_time)
}

df1 = pd.DataFrame(data1)
df1.set_index('index', inplace=True)   #name 열을 인덱스로 지정


df2 = pd.DataFrame(data2)
df2.set_index('index', inplace=True)   #name 열을 인덱스로 지정

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2]).to_csv('tec2result.csv') #csv파일로 생성